
# LSST Twilight Planner (Modular) — Notebook

This notebook uses the split package `twilight_planner_pkg` to plan twilight observations.
Modify the config cell to adjust filters, exposure times, and nightly caps.


In [4]:

# If you don't see the package, make sure /mnt/data is on sys.path
import sys, os, pandas as pd
sys.path.append('./')

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# (Optional) Quick check that files exist
csv_path = '../data/ATLAS_2021_to25_cleaned.csv'  # change if needed
outdir = '../twilight_outputs'
os.makedirs(outdir, exist_ok=True)

print('CSV exists:', os.path.exists(csv_path))
print('Output dir:', outdir)


CSV exists: True
Output dir: ../twilight_outputs



## Configure the site and planner knobs
- **Site**: LSST @ Cerro Pachón (approx.): lat = -30.2446°, lon = -70.7494°, height = 2663 m  
- **Filters**: default `g,r,i,z` with 5 s exposure each (edit below)  
- **Caps**: per-SN cap 120 s, morning/evening window cap 600 s each  
- **Visibility**: minimum altitude 20°; moon separation handled per-filter when `require_single_time_for_all_filters=True`


In [5]:

# ---- User-editable parameters ----
LAT_DEG = -30.2446
LON_DEG = -70.7494
HEIGHT_M = 2663

START_DATE = "2024-01-01"   # UTC
END_DATE   = "2024-01-03"   # UTC

FILTERS = ["g","r","i","z"]
EXPOSURE_BY_FILTER = {"g":5.0, "r":5.0, "i":5.0, "z":5.0}

MIN_ALT_DEG = 20.0
EVENING_CAP_S = 600.0
MORNING_CAP_S = 600.0
PER_SN_CAP_S = 120.0
MAX_SN_PER_NIGHT = 10
TWILIGHT_STEP_MIN = 2
REQUIRE_SINGLE_TIME_FOR_ALL = True

cfg = PlannerConfig(
    lat_deg=LAT_DEG, lon_deg=LON_DEG, height_m=HEIGHT_M,
    min_alt_deg=MIN_ALT_DEG,
    filters=FILTERS,
    exposure_by_filter=EXPOSURE_BY_FILTER,
    twilight_step_min=TWILIGHT_STEP_MIN,
    evening_cap_s=EVENING_CAP_S,
    morning_cap_s=MORNING_CAP_S,
    per_sn_cap_s=PER_SN_CAP_S,
    max_sn_per_night=MAX_SN_PER_NIGHT,
    require_single_time_for_all_filters=REQUIRE_SINGLE_TIME_FOR_ALL,
)

cfg


PlannerConfig(lat_deg=-30.2446, lon_deg=-70.7494, height_m=2663, min_alt_deg=20.0, typical_days_by_type={}, default_typical_days=30, slew_small_deg=3.0, slew_small_time_s=2.0, slew_rate_deg_per_s=3.5, slew_settle_s=2.0, readout_s=2.0, filter_change_s=120.0, filters=['g', 'r', 'i', 'z'], exposure_by_filter={'g': 5.0, 'r': 5.0, 'i': 5.0, 'z': 5.0}, carousel_capacity=6, twilight_step_min=2, evening_cap_s=600.0, morning_cap_s=600.0, max_sn_per_night=10, per_sn_cap_s=120.0, min_moon_sep_by_filter={'g': 30.0, 'r': 25.0, 'i': 20.0, 'z': 15.0}, require_single_time_for_all_filters=True, ra_col=None, dec_col=None, disc_col=None, name_col=None, type_col=None)


## Run the planner
This will produce two CSVs:
- `lsst_twilight_plan_*.csv` — per-SN schedule
- `lsst_twilight_summary_*.csv` — per-night summary


In [6]:

perSN_df, nights_df = plan_twilight_range_with_caps(
    csv_path=csv_path,
    outdir=outdir,
    start_date=START_DATE,
    end_date=END_DATE,
    cfg=cfg,
    verbose=True,
)

print("Per-SN rows:", len(perSN_df), "  Nights summary rows:", len(nights_df))
perSN_df.head(10)


RA raw (numeric) range:  min=0.043167, max=359.855458
Dec raw (numeric) range: min=-87.141200, max=87.960305


Nights:   0%|          | 0/3 [00:00<?, ?night/s]

2024-01-01: eligible=81 visible=62 planned_total=9
2024-01-02: eligible=85 visible=71 planned_total=7
2024-01-03: eligible=86 visible=74 planned_total=7
Wrote:
  ../twilight_outputs/lsst_twilight_plan_2024-01-01_to_2024-01-03.csv
  ../twilight_outputs/lsst_twilight_summary_2024-01-01_to_2024-01-03.csv
Rows: per-SN=23, nights*windows=9
Per-SN rows: 23   Nights summary rows: 9


,date,twilight_window,SN,RA_deg,Dec_deg,best_twilight_time_utc,best_alt_deg,filters,exposure_s,readout_s,filter_changes_s,slew_s,total_time_s
0,2024-01-01,morning,2023yxi,47.417777,-31.222880,2024-01-01T01:12:00+00:00,89.09,g,5.0,2.0,0.0,0.0,7.0
1,2024-01-01,morning,2023yqc,38.192499,-38.310448,2024-01-01T00:36:00+00:00,82.04,g,5.0,2.0,0.0,6.1,13.1
2,2024-01-01,morning,2023zzn,46.861701,-42.658672,2024-01-01T01:10:00+00:00,77.67,g,5.0,2.0,0.0,5.4,12.4
3,2024-01-01,morning,2023ypx,18.428125,-41.627731,2023-12-31T23:44:00+00:00,77.51,g,5.0,2.0,0.0,9.1,16.1
4,2024-01-01,morning,2023ywq,15.922583,-39.874981,2023-12-31T23:44:00+00:00,77.89,g,5.0,2.0,0.0,4.0,11.0
5,2024-01-01,morning,2023zrc,17.044547,-36.059907,2023-12-31T23:44:00+00:00,81.17,g,5.0,2.0,0.0,4.3,11.3
6,2024-01-01,evening,2024C,145.975500,-29.727481,2024-01-01T08:09:00+00:00,85.01,g,5.0,2.0,0.0,0.0,7.0
7,2024-01-01,evening,2023zce,141.587417,-31.226239,2024-01-01T08:09:00+00:00,81.23,g,5.0,2.0,0.0,4.3,11.3
8,2024-01-01,evening,2023zea,148.281780,-37.998441,2024-01-01T08:09:00+00:00,81.64,g,5.0,2.0,0.0,5.6,12.6
9,2024-01-02,morning,2023yqc,38.192499,-38.310448,2024-01-02T00:32:00+00:00,82.04,g,5.0,2.0,0.0,0.0,7.0


In [7]:

# Show the nightly summary
nights_df


,date,twilight_window,n_candidates,n_planned,sum_time_s,window_cap_s
0,2024-01-01,morning,6,6,70.9,600
1,2024-01-01,evening,3,3,30.9,600
2,2024-01-01,W2,1,0,0.0,0
3,2024-01-02,morning,3,3,33.3,600
4,2024-01-02,evening,4,4,42.4,600
5,2024-01-02,W2,3,0,0.0,0
6,2024-01-03,morning,3,3,37.3,600
7,2024-01-03,evening,4,4,42.4,600
8,2024-01-03,W2,3,0,0.0,0



## Inspect outputs on disk


In [8]:

import glob, os, pandas as pd

files = sorted(glob.glob(os.path.join(outdir, "lsst_twilight_*.csv")))
files


['../twilight_outputs/lsst_twilight_plan_2024-01-01_to_2024-01-03.csv',
 '../twilight_outputs/lsst_twilight_summary_2024-01-01_to_2024-01-03.csv']